# Deer Hunter

By tradycji Funduszu stało się zadość, skontruujemy grę dawno temu wymyśloną przez jednego z uczestników - Polowanie na Jelenie. Roboczo nazwijmy tę grę Deer Hunter.

Tym razem oszczędzimy sobie minimalistycznego wyglądu Ponga i skorzystamy z gotowych tekstur, by urozmaicić trochę wygląd gry. Poniżej zamieszczę tekstury z których będziemy korzystać przy produkcji.

[las](./data/forest.png)
[jeleń](./data/deer.png)
[celownik](./data/crosshair.png)
[ziemia](./data/ground.png)
[kamienie](./data/rock.png)
[drzewa](./data/trees.png)


## Zaczynamy!

Ponownie ustawimy nasz projekt na dwuwymiarowy. Następnie ustawimy plan gry tak, by tło lasu było kompletnie z tyłu, natomiast drzewa były bliżej, zasłaniając nasze jelenie. Będziemy potrzebowali trzech skryptów: `CrosshairScript`, `DeerScript`, `GameManagerScript`. Będą odpowiadały za:

- `CrosshairScript` - celownik podąża za kursorem myszy i określa, czy wskazuje na jelenia czy nie. W przypadku kliknięcia wysyła sygnał do obiektu by go zestrzelić
- `DeerScript` - odpowiada za zachowanie jelenia i jego ruch na planszy
- `GameManagerScript` - odpowiada za przeliczanie punktów i generowanie nowych jelonków na planszy. W przypadku zestrzelenia jelenia jest informowany o tym by doliczyć kolejny punkt i stworzyć nowy element. W przypadku ucieczki jelonka poza planszę jest również informowany o tym, by obniżyć liczbę punktów.

## Proces tworzenia - Jeleń

Stworzymy teraz obiekt jelenia. Zaczniemy prosto, od przeciągnięcia tekstury na Scenę - w ten sposób stworzymy obiekt który będzie nam reprezentował zwierzaka. Następnie, dodamy mu następujące komponenty:

- `PolygonCollider2D` - dla detekcji strzału, ograniczonej DOKŁADNIE DO KSZTAŁTU OBRAZKA.
- `RigidBody2D`, KONIECZNIE z zaznaczonym Body Type : Kinematic. Pozwoli nam to na nadanie jednostajnego ruchu jelonkowi, z jednoczesnym wykluczeniem kolizji o inne obiekty (np. drzewa).

![](./data/rigidbody2d.png)

- `DeerScript` - stworzony przez nas skrypt generowania i ruchu jelenia po planszy. Starałem się ograniczyć go do minimum, by nie musiał oddziaływać z innymi skryptami.

```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class DeerScript : MonoBehaviour
{
    public float MinimalHeight = 0;
    public float MaximalHeight = 5;
    public float StartingLocation = 15;
    public float MinimalSpeed = 5;
    public float MaximalSpeed = 10;
    public float MinimalSize = 0.1f;
    public float MaximalSize = 1;

    public float KillDistance = 20;
    // Start is called before the first frame update
    void Start()
    {
        var random = Random.Range(0, 1024);
        var height = Random.Range(MinimalHeight, MaximalHeight);
        var speed = Random.Range(MinimalSpeed, MaximalSpeed);
        var size = Random.Range(MinimalSize, MaximalSize);
        transform.position = new Vector2(StartingLocation, height);
        transform.localScale = Vector3.one * size;

        if (random > 512)
        {
            var pos = transform.position;
            pos.x = -pos.x;
            transform.position = pos;
            var scale = transform.localScale;
            scale.x = -scale.x;
            transform.localScale = scale;
            speed = -speed;
        }

        GetComponent<Rigidbody2D>().velocity = Vector2.left * speed;
    }

    // Update is called once per frame
    void Update()
    {
        if (Mathf.Abs(transform.position.x) > KillDistance)
        {
            Destroy(gameObject);
        }
    }
}
```

Następnie stworzony obiekt Jelenia przeniesiemy do ekranu Projektu, by móc z niego korzystać w przyszłości.

## Proces tworzenia - Game Manager

Dodamy teraz kilka kolejnych elementów. Warto byłoby stworzyć zapis punktów + system który podliczałby te punkty w czasie, tworzył kolejne jelenie, itp itd. Do obiektu Main Camera dodamy teraz element Text i zmienimy go tak, by w sensowny sposób pokazywał nam liczbę punktów:

![](./data/text-ui.png)

Następnie stworzymy pusty element `Game Manager` do którego dodamy komponent skryptu `GameManagerScript`. Wewnątrz pliku C# dodamy następujący kod źródłowy:

```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using UnityEngine.UI;

public class GameManagerScript : MonoBehaviour
{
    // Start is called before the first frame update
    void Start()
    {
        
    }

    public List<GameObject> DeerList = new List<GameObject>();
    public int MaxDeerCount = 1;
    public int Score = 0;
    public GameObject DeerObject;


    // Update is called once per frame
    void Update()
    {
        var newList = new List<GameObject>();
        foreach (var deer in DeerList)
        {
            if (deer == null)
            {
                Score--;
                if (Score < 0) Score = 0;
                continue;
            }
            newList.Add(deer);
        }

        DeerList = newList;

        if (DeerList.Count < MaxDeerCount)
        {
            var deer = Instantiate(DeerObject);
            DeerList.Add(deer);
        }

        var text = GameObject.Find("Text");
        if (text)
        {
            text.GetComponent<Text>().text = Score.ToString();
        }

    }

    public void KillDeer(GameObject deer)
    {
        Score++;
        DeerList.Remove(deer);
        Destroy(deer);
    }
}

```

## Proces tworzenia - Celownik

Ostatni obiekt jaki dodamy to celownik - będzie on podążał za kursorem naszej myszy (de facto będzie kursorem myszy) i zajmował się namierzaniem i zestrzeliwywaniem jeleni. Tworzymy go poprzez przeciągnięcie tekstury Crosshair na Scenę i dodanie do niej komponentu `CrosshairScript`. Do owego komponentu dodajemy następujący kod źródłowy:

```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class CrosshairScript : MonoBehaviour
{
    // Start is called before the first frame update
    void Start()
    {
        Cursor.visible = false;
    }

    // Update is called once per frame
    void Update()
    {
        var pos = Camera.main.ScreenToWorldPoint(Input.mousePosition);
        var crosshairPos = transform.position;
        crosshairPos.x = pos.x;
        crosshairPos.y = pos.y;
        transform.position = crosshairPos;

        if (Input.GetMouseButtonUp(0))
        {

            var hit = Physics2D.Raycast(pos, Vector2.zero, Mathf.Infinity);

            if (hit)
            {
                var deer = hit.collider.gameObject;
                var manager = GameObject.Find("GameManager");
                if (deer.GetComponent<DeerScript>() && manager)
                {
                    manager.GetComponent<GameManagerScript>().KillDeer(deer);
                }
            }
        }
    }
}

```

### Dodatek - port na Androida

![](./data/android.jpg)

### Następne zajęcia - funduszowy Flappy Bird?